In [2]:
!pip install torch torchvision efficientnet_pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16458 sha256=ae358b3257993e2b40e9fa4a0175c9cc90f4443a87a6e6937437a797281e5012
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\8b\6f\9b\231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
Successfully built efficientnet_pytorch


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
# ImageNet normalization + augmentations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.RandomHorizontalFlip(),  # Augment data
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load your dataset
train_dataset = datasets.ImageFolder(root=r'C:\Users\DELL\OneDrive\Desktop\Minorsem6Data\Augmented Data', transform=transform)
val_dataset = datasets.ImageFolder(root=r'C:\Users\DELL\OneDrive\Desktop\Minorsem6Data\Raw Data', transform=transform)

# DataLoader (batching & shuffling)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [7]:
# Load pre-trained EfficientNet-B0
model = EfficientNet.from_pretrained('efficientnet-b0')

# Modify the last layer for 26 classes
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 26)

# Move model to GPU if available
model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:03<00:00, 5.87MB/s]


Loaded pretrained weights for efficientnet-b0


In [8]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower LR for transfer learning

In [9]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()  # Set to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/1, Loss: 0.8660
